In [1]:
import torch
import cv2
from torchvision import models, transforms
import serial
import serial.tools.list_ports
import time
import math


In [2]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from torch import nn
import numpy as np

ser = serial.Serial('/dev/cu.usbserial-144140', 115200,timeout=1)
num=0 
label_name_list = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
save_path = "resnet18v1.pth"
num_classes = len(label_name_list)
img_width = 224

# 定义一个预测函数
# 加载预训练的 ResNet-18 模型
model_reload = models.resnet18(pretrained=True)
num_ftrs = model_reload.fc.in_features
model_reload.fc = nn.Linear(num_ftrs, num_classes)
# model_reload.to('cpu')
model_reload.load_state_dict(torch.load(save_path,map_location=torch.device('cpu')))

/Users/macbookpro/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/macbookpro/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [3]:
# 图像预处理
# ori = image.copy()
pred_transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize((img_width,img_width)),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                    ])



# 定义一个预测函数
def predict(image):

    model_reload.eval()
    # 进行预测
    with torch.no_grad():
        outputs = model_reload(image)
        _, preds = torch.max(outputs, 1)
        # 计算预测概率
        pred_score_list = nn.functional.softmax(outputs[0], dim=0).cpu().numpy()
        # 获取最大值
        pred_score = np.max(pred_score_list)

    # 预测最可能的类别
    predicted_idx = str(preds[0].item())
    label_name = label_name_list[preds[0].item()]
    return label_name,str(pred_score)  # 返回预测结果和处理后的图像

In [4]:

# 使用OpenCV捕获摄像头视频
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: failed to capture image")
        break

    # 将BGR图像转换为RGB，然后应用预处理
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = pred_transform(rgb_frame).unsqueeze(0)  # 添加批处理维度
    #img_tensor = transform(rgb_frame)
    #img_tensor = img_tensor.unsqueeze(0)  # 添加批次维度
    label_name,pred_score =  predict(image)



    # 在帧上显示预测结果
    cv2.putText(frame, label_name, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    # print ("label_name >",label_name)
    ser.write(label_name.encode('utf-8'))
    #comdata=label_name+'0'
    #ser.write(comdata.encode('utf-8'))
    #num=num+1
    #if num==10:
     #   comdata=label_name+'0'
     #   num=0
     #   ser.write(comdata.encode('utf-8'))
        # ser.flushOutput()  # 清空发送缓存
    time.sleep(0.1)
    # 显示帧
    cv2.imshow('frame', frame)

    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

    if cv2.waitKey(1)  == 27:
        break
# 释放摄像头资源
cap.release()
cv2.destroyAllWindows()

In [5]:
# ser = serial.Serial('/dev/cu.usbserial-143140', 9600) 
# def send_data(data):
#     ser.write(data.encode())

: 